Purpose:
Retrieve full list of UK stations from http://orr.gov.uk/statistics/published-stats/station-usage-estimates

Create tables in the database and add data

Programmer:  Robin Linacre
Last updated: 30/04/2016

In [1]:
import pandas as pd
pd.options.display.max_columns = 999

In [2]:
# stations_df = pd.read_excel("http://orr.gov.uk/__data/assets/excel_doc/0019/20179/Estimates-of-Station-Usage-in-2014-15.xlsx",2)
stations_df = pd.read_excel("http://www.orr.gov.uk/__data/assets/excel_doc/0020/23357/estimates-of-station-usage-2015-16.xlsx", 2)

In [3]:
stations_df.head()


,NLC,TLC,Station Name,Region,Local Authority,Constituency,OS Grid Easting,OS Grid Northing,Station Facility Owner,Station Group,PTE Urban Area Station,London Travelcard Area,SRS Code,SRS Description,NR Route,CRP Line Designation,Entries_Full,Entries_Reduced,Entries_Season,Entries_Total,Exits_Full,Exits_Reduced,Exits_Season,Exits_Total,1516 Entries & Exits,Estimated absolute change in Usage due to 2015/16 London Travelcard Methodology,1415 Entries & Exits,1516 Interchanges,Estimated absolute change in Interchanges due to 2015/16 London Travelcard Methodology,Large station change flag,Small station change flag,Explanation of large change,Source for explanation of large change
0,5131.0,ABW,Abbey Wood,London,Bexley (London),Erith and Thamesmead,547374,179035,Southeastern,NaN,NaN,London Travelcard Area Station,A.07,Dartford Lines to Gravesend and Hayes Branch,South East,NaN,319067,384846,760823,1464736,319067,384846,760823,1464736,2929472,-390349,3319408,0,0,1.0,0.0,London Stations Methodology Change,NaN
1,3813.0,ABE,Aber,Wales - Cymru,Caerffili - Caerphilly,Caerphilly,314870,186950,Arriva Trains Wales,NaN,NaN,NaN,L.18,South Wales Valleys,Wales,NaN,47559,3054,56885,107498,47559,3054,56885,107498,214996,0,212546,0,0,0.0,0.0,NaN,NaN
2,3801.0,ACY,Abercynon,Wales - Cymru,"Rhondda, Cynon, Taf - Rhondda, Cynon, Taff",Cynon Valley,308278,194824,Arriva Trains Wales,NaN,NaN,NaN,L.18,South Wales Valleys,Wales,NaN,76547,7170,53985,137702,76547,7170,53985,137702,275404,0,265458,41720.1,0,0.0,0.0,NaN,NaN
3,3982.0,ABA,Aberdare,Wales - Cymru,"Rhondda, Cynon, Taf - Rhondda, Cynon, Taff",Cynon Valley,300400,202800,Arriva Trains Wales,NaN,NaN,NaN,I.99,Other Freight Lines,Wales,NaN,167408,19463,96581,283452,167408,19463,96581,283452,566904,-1,551928,0,0,0.0,0.0,NaN,NaN
4,8976.0,ABD,Aberdeen,Scotland,Aberdeen City,Aberdeen North Burgh,394127,805872,ScotRail,NaN,NaN,NaN,P.09,Dundee - Aberdeen,Scotland,NaN,600344,846508,283120,1729972,600344,846508,283120,1729972,3459944,0,3742646,204139,0,0.0,0.0,NaN,NaN


In [4]:
headers = [h.lower() for h in list(stations_df.columns)]
headers = [h.replace(" ", "_").replace("(", "").replace(")","") for h in headers]
stations_df.columns = headers
stations_df.head()
stations_df = stations_df[pd.notnull(stations_df["nlc"])]

In [5]:
stations_df["nlc"] = stations_df["nlc"].astype(int)

In [6]:
# Now write out to postgres
from mylibrary.connections import conn, engine, cursor
stations_df.to_sql("all_stations", engine, schema="tt_h", if_exists="replace", index=False)

/Users/robinlinacre/anaconda/lib/python3.6/site-packages/sqlalchemy/dialects/postgresql/base.py:2526: SAWarning: Did not recognize type 'geometry' of column 'geom'
  (attype, name))


In [7]:
# Create geometry column for the points including a spatial index for efficient querying

sql = """
SELECT AddGeometryColumn ('tt_h', 'all_stations', 'geom', 27700, 'POINT', 2);
UPDATE tt_h.all_stations SET geom = ST_GeomFromText('POINT(' || os_grid_easting || ' ' || os_grid_northing || ')', 27700 );
CREATE INDEX idx_geom_all_stations_points ON tt_h.all_stations USING gist(geom);
"""
cursor.execute(sql)
conn.commit()

In [8]:
#Make a lat and lng column

sql = """

ALTER TABLE tt_h.all_stations ADD lat float, ADD lng float, ADD icscode text, 
ADD icscode_status text, ADD tfl_request text, ADD tfl_response json, ADD tfl_message  text;
UPDATE tt_h.all_stations SET
    lng = ST_X(ST_TRANSFORM(geom, 4326)),
    lat = ST_Y(ST_TRANSFORM(geom,4326));
ALTER TABLE tt_h.all_stations  ADD PRIMARY KEY (nlc);
""" 
cursor.execute(sql)
conn.commit()

In [9]:

sql = """
select * from tt_h.all_stations
"""
pd.read_sql(sql, conn)

,nlc,tlc,station_name,region,local_authority,constituency,os_grid_easting,os_grid_northing,station_facility_owner,station_group,pte_urban_area_station,london_travelcard_area,srs_code,srs_description,nr_route,crp_line_designation,entries_full,entries_reduced,entries_season,entries_total,exits_full,exits_reduced,exits_season,exits_total,1516_entries_&_exits,estimated_absolute_change_in_usage_due_to_2015/16_london_travel,1415_entries_&_exits,1516_interchanges,estimated_absolute_change_in_interchanges_due_to_2015/16_london,large_station_change_flag,small_station_change_flag,explanation_of_large_change,source_for_explanation_of_large_change,geom,lat,lng,icscode,icscode_status,tfl_request,tfl_response,tfl_message
0,8726,FOC,Falls Of Cruachan,Scotland,Argyll and Bute,Argyll and Bute,207990,726750,ScotRail,None,None,None,Q.14,West Highland Line,Scotland,None,229,138,0,367,229,138,0,367,734,0,654,0,0,0.0,0.0,None,None,0101000020346C000000000000B063094100000000BC2D...,56.393882,-5.112478,None,None,None,None,None
1,5131,ABW,Abbey Wood,London,Bexley (London),Erith and Thamesmead,547374,179035,Southeastern,None,None,London Travelcard Area Station,A.07,Dartford Lines to Gravesend and Hayes Branch,South East,None,319067,384846,760823,1464736,319067,384846,760823,1464736,2929472,-390349,3319408,0,0,1.0,0.0,London Stations Methodology Change,None,0101000020346C0000000000005CB4204100000000D8DA...,51.491061,0.121394,None,None,None,None,None
2,3813,ABE,Aber,Wales - Cymru,Caerffili - Caerphilly,Caerphilly,314870,186950,Arriva Trains Wales,None,None,None,L.18,South Wales Valleys,Wales,None,47559,3054,56885,107498,47559,3054,56885,107498,214996,0,212546,0,0,0.0,0.0,None,None,0101000020346C000000000000D83713410000000030D2...,51.574964,-3.229828,None,None,None,None,None
3,3801,ACY,Abercynon,Wales - Cymru,"Rhondda, Cynon, Taf - Rhondda, Cynon, Taff",Cynon Valley,308278,194824,Arriva Trains Wales,None,None,None,L.18,South Wales Valleys,Wales,None,76547,7170,53985,137702,76547,7170,53985,137702,275404,0,265458,41720.0864,0,0.0,0.0,None,None,0101000020346C000000000000D8D012410000000040C8...,51.644707,-3.326988,None,None,None,None,None
4,3982,ABA,Aberdare,Wales - Cymru,"Rhondda, Cynon, Taf - Rhondda, Cynon, Taff",Cynon Valley,300400,202800,Arriva Trains Wales,None,None,None,I.99,Other Freight Lines,Wales,None,167408,19463,96581,283452,167408,19463,96581,283452,566904,-1,551928,0,0,0.0,0.0,None,None,0101000020346C000000000000C05512410000000080C1...,51.715056,-3.443083,None,None,None,None,None
5,8976,ABD,Aberdeen,Scotland,Aberdeen City,Aberdeen North Burgh,394127,805872,ScotRail,None,None,None,P.09,Dundee - Aberdeen,Scotland,None,600344,846508,283120,1729972,600344,846508,283120,1729972,3459944,0,3742646,204138.8141,0,0.0,0.0,None,None,0101000020346C0000000000003C0E184100000000E097...,57.143703,-2.098685,None,None,None,None,None
6,9090,AUR,Aberdour,Scotland,Fife,Kirkcaldy and Cowdenbeath,319100,685400,ScotRail,None,None,None,P.06,Fife Loop,Scotland,None,19593,30571,13492,63656,19593,30571,13492,63656,127312,0,129474,0,0,0.0,0.0,None,None,0101000020346C000000000000F079134100000000B0EA...,56.054590,-3.300560,None,None,None,None,None
7,5538,BDH,Bedhampton,South East,Havant,Havant,470781,106529,South West Trains,None,None,None,C.06,Woking - Portsmouth,Wessex,None,22504,17762,26197,66463,22504,17762,26197,66463,132926,6,136952,0,0,0.0,0.0,None,None,0101000020346C000000000000F4BB1C41000000001002...,50.853955,-0.995828,None,None,None,None,None
8,3005,CME,Combe,South East,West Oxfordshire,Witney,441850,215060,Great Western Railway,None,None,None,J.11,Cotswolds Line,Western,None,591,77,62,730,591,77,62,730,1460,2,1248,0,0,0.0,0.0,None,None,0101000020346C000000000000E8F71A4100000000A040...,51.832592,-1.394073,None,None,None,None,None
9,7201,LAK,Lakenheath,East,Forest Heath,West Suffolk,572390,286293,East Anglia,None,None,None,D.09,Ely - Norwich,Anglia,None,66,126,16,208,66,126,16,208,416,0,458,0,0,0.0,0.0,None,None,0101000020346C000000000000CC772141000000005479...,52.